# Preprocess [Thai Common Voice Corpus 7.0](https://commonvoice.mozilla.org/en/datasets)

Notebook by [@tann9949](https://github.com/tann9949).

In [ ]:
# !pip install pydub
# !pip install pythainlp==2.3.1
# !pip install ipywidgets

In [1]:
import os
import re
from typing import List, Dict, Tuple

import pandas as pd
from scipy.io import wavfile
from pythainlp.tokenize import word_tokenize

from spell_correction import correct_sentence

# ipython
import IPython.display as ipd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

In [2]:
cv_root: str = "../data/cv-corpus-7.0-2021-07-21"

train: pd.DataFrame = pd.read_csv(f"{cv_root}/th/train.tsv", delimiter="\t")
train["set"] = "train"
dev: pd.DataFrame = pd.read_csv(f"{cv_root}/th/dev.tsv", delimiter="\t")
dev["set"] = "dev"
test: pd.DataFrame = pd.read_csv(f"{cv_root}/th/test.tsv", delimiter="\t")
test["set"] = "test"
data: pd.DataFrame = pd.read_csv(f"{cv_root}/th/validated.tsv", delimiter="\t")

command: str = "sox {mp3_path} -t wav -r {sr} -c 1 -b 16 - |"

In [3]:
def get_full_path(cv_root: str, path: str) -> str:
    """Get full path from `path` instance in cv data"""
    f_path: str = f"{cv_root}/th/clips/{path}"
    if not os.path.exists(f_path):
        raise FileNotFoundError(f"File `{f_path}` does not exists")
    return f_path

def get_char(texts: List[str]) -> List[str]:
    """Get unique char from list of documents"""
    return sorted(set([char for sent in texts for char in sent]))


def get_audio_len(audio_path: str) -> float:
    """Get audio duration in second"""
    sr, wav = wavfile.read(audio_path.replace("mp3", "wav").replace("clips", "wav"))
    return len(wav) / sr
#     audio: AudioSegment = AudioSegment.from_mp3(audio_path)
#     return len(audio) / 1000  # pydub duration works in ms


def sec_to_hour(second: int) -> str:
    """Convert second to XXH:YYm:ZZs format"""
    minute, second = divmod(second, 60)
    hour, minute = divmod(minute, 60)
    return f"{int(hour)}H:{int(minute)}m:{second:.2f}s"

In [8]:
# Rules mapping obtained from exploring data
mapping_char: Dict[str, str] = {
    r"!": " ",
    r'"': " ",
    r"'": " ",
    r",": " ",
    r"-": " ",
    r"\.{2,}": " ",
    r"\.$": "",  # full stop at end of sentence
    r"([ก-์])\.([ก-์])": r"\1. \2",  # บจก.XXX -> บจก. XXX
    r":": " ",
    r";": " ",
    r"\?": " ",
    r"‘": " ", 
    r"’": " ",
    r"“": " ", 
    r"”": " ",
    r"~": " ",
    r"—": " ",
    r"\.": " ",
}

# text that needs to be fixed
change_text: Dict[str, str] = {
    "common_voice_th_26103939.mp3": "บริษัทจำกัดดาบเพ็ชร์",
    "common_voice_th_27269555.mp3": "ศุนย์การค้า เจ พี ไรวา",
    "common_voice_th_26429486.mp3": "โศรดาพลัดถิ่น ชอุ่ม ปัญจพรรค์",
    "common_voice_th_25668677.mp3": "มิสเตอร์ ลินคอล์น",
    "common_voice_th_25677501.mp3": "โอ้พระเจ้าพวกเขาฆ่า เคนนี่",
    "common_voice_th_25696778.mp3": "ฉันสงสัยว่า ซี เซคชั่น จะได้รับความนิยมมากกว่าการเกิดตามธรรมชาติในวันหนึ่ง",
    "common_voice_th_25728649.mp3": "เนื่องจากการขาดโปรแกรมความผิดพลาด โจฮันน่า จึงตัดสินใจขายการหาประโยชน์ในตลาดมืด",
    "common_voice_th_25700969.mp3": "บรูค วิจารณ์ตัวเองเพราะรอยยิ้ม",
    "common_voice_th_23897115.mp3": "กลุ่มอาการ แอสเพอร์เจอร์ เป็นรูปแบบของออทิสติก",
    "common_voice_th_25705973.mp3": "การเปิดใช้งาน ซอฟต์แม็ก นั้นมีราคาแพงมากเพื่อใช้ในการคำนวณ",
    "common_voice_th_24149507.mp3": "แอสโทเทริฟ ถูกใช้ปูพื้นในสนามเด็กเล่นกลางแจ้ง",
    "common_voice_th_25665768.mp3": "เฟสบุ๊ก รวบรวมข้อมูลเกี่ยวกับผู้ที่ไม่ได้เป็นสมาชิก",
    "common_voice_th_25636404.mp3": "ใครอยู่ ม โปรดพาอ้อมไปกินข้าวที",
    "common_voice_th_25903042.mp3": "ฉันคิดว่ามันพร้อมแล้ว ฉันกล่าว",
    "common_voice_th_26701409.mp3": "จอมพล ป มีแนวคิดว่าราษฎรจะรักชาติของตนมิได้",
    "common_voice_th_26147573.mp3": "จอมพล ป พิบูลสงคราม",
    "common_voice_th_25900743.mp3": "ไม่ใช่เพื่อคุณ กัซซี่กล่าว",
    "common_voice_th_25682485.mp3": "ขอขอบคุณ จะตรวจสอบอย่างแน่นอน",
    "common_voice_th_25885519.mp3": "ไม่ใช่อะไรสลักสำคัญ ทหารตอบกลับ",
    "common_voice_th_25899784.mp3": "ไม่ใช่อย่างน้อยที่สุด เขากล่าว",
    "common_voice_th_25704992.mp3": "การวินิจฉัยโรคจำเป็นต้องทำอย่างละเอียดและรอบคอบ", # การวินิจฉัยโรคจําเป็นต้องทําอย่างละเอียดและรอบคอบ
    "common_voice_th_25628971.mp3": "เขากำลังทำงานอยู่",  # "เขากำลังทํางานอยู่"
}

# sentence to skip
skip_sentence: Dict[str, str] = {
    "common_voice_th_25683553.mp3": "บางครั้งคนอังกฤษก็ใช้คำว่า whilst แม้ว่ามันจะดูเก่าไปแล้วก็ตาม",
    "common_voice_th_25682645.mp3": "ฉันใช้Flickr ในการเก็บภาพถ่ายออนไลน์ที่สำคัญของฉันเป็นส่วนใหญ่",
    "common_voice_th_25673002.mp3": "Fury X เป็นการ์ดกราฟิกที่ทรงพลังมาก",
    "common_voice_th_25695730.mp3": "Brexiteers เป็นชื่อเล่นที่มอบให้กับผู้ที่สนับสนุนการลงประชามติของ Brexit"
}

skip_rules: List[str] = [
    r"[a-zA-Z]",
]

## Preprocess Data

In [13]:
def preprocess_text(text: str) -> str:
    """Preprocess text according to `mapping_char`"""
    text = correct_sentence(text)
    for pattern, sub in mapping_char.items():
        text = re.sub(pattern, sub, text)
    text = re.sub(r" +", " ", text)  # merge multiple whitespaces to one
    return text

def is_valid(text: str) -> bool:
    """Validate each utterance"""
    for rule in skip_rules:
        if re.search(rule, text):
            return False
        else:
            return True

In [16]:
texts: pd.DataFrame = data[["path", "sentence"]]
texts["path"] = texts["path"].map(lambda x: get_full_path(cv_root, x))
texts = texts[texts["sentence"].apply(lambda x: is_valid(x))].reset_index(drop=True)
texts["sentence"] = texts["sentence"].map(lambda x: preprocess_text(x))

remove_idx: List[int] = []
for i in tqdm(range(len(texts)), total=len(texts)):
    f_path: str = os.path.basename(texts.loc[i, "path"])
    for name, change in change_text.items():
        if f_path in name:
            print(texts.loc[i, "path"])
            print("\tReplacing", f"`{texts.loc[i, 'sentence']}`...", "with", f"`{change}`...")
            texts.loc[i, "sentence"] = change
            break
    for name in skip_sentence.keys():
        if f_path in name:
            print(texts.loc[i, "path"])
            print(f"\tRemoving `{texts.loc[i, 'sentence']}`...")
            remove_idx.append(i)
            
texts = texts.drop(texts.index[remove_idx])
            
texts["path"] = texts["path"].map(lambda x: x.replace(f"{cv_root}/th/clips/", ""))
texts = texts.reset_index(drop=True)
texts["sentence"] = texts["sentence"].map(lambda x: re.sub(r" +", " ", x).strip())

/home/cstorm125/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
Correct ํ + า => ำ
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
Correct ํ + า => ำ
ๆ Rep

ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
Correct ํ + า => ำ
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ Replaced
ๆ 

../data/cv-corpus-7.0-2021-07-21/th/clips/common_voice_th_25704992.mp3
	Replacing `การวินิจฉัยโรคจำเป็นต้องทำอย่างละเอียดและรอบคอบ`... with `การวินิจฉัยโรคจำเป็นต้องทำอย่างละเอียดและรอบคอบ`...
../data/cv-corpus-7.0-2021-07-21/th/clips/common_voice_th_25628971.mp3
	Replacing `เขากำลังทำงานอยู่`... with `เขากำลังทำงานอยู่`...
../data/cv-corpus-7.0-2021-07-21/th/clips/common_voice_th_25636404.mp3
	Replacing `ใครอยู่ม โปรดพาอ้อมไปกินข้าวที`... with `ใครอยู่ ม โปรดพาอ้อมไปกินข้าวที`...
../data/cv-corpus-7.0-2021-07-21/th/clips/common_voice_th_26147573.mp3
	Replacing `จอมพล ป พิบูลสงคราม`... with `จอมพล ป พิบูลสงคราม`...
../data/cv-corpus-7.0-2021-07-21/th/clips/common_voice_th_25682485.mp3
	Replacing `ขอขอบคุณ จะตรวจสอบอย่างแน่นอน`... with `ขอขอบคุณ จะตรวจสอบอย่างแน่นอน`...
../data/cv-corpus-7.0-2021-07-21/th/clips/common_voice_th_25885519.mp3
	Replacing `ไม่ใช่อะไรสลักสำคัญ ทหารตอบกลับ`... with `ไม่ใช่อะไรสลักสำคัญ ทหารตอบกลับ`...
../data/cv-corpus-7.0-2021-07-21/th/clips/common_voice_th_2

In [17]:
_ = [print(f"\"{c}\", ", end="") for c in get_char([x[1] for x in texts.values.tolist()])]

" ", "[", "]", "ก", "ข", "ฃ", "ค", "ฆ", "ง", "จ", "ฉ", "ช", "ซ", "ฌ", "ญ", "ฎ", "ฏ", "ฐ", "ฑ", "ฒ", "ณ", "ด", "ต", "ถ", "ท", "ธ", "น", "บ", "ป", "ผ", "ฝ", "พ", "ฟ", "ภ", "ม", "ย", "ร", "ฤ", "ล", "ว", "ศ", "ษ", "ส", "ห", "ฬ", "อ", "ฮ", "ฯ", "ะ", "ั", "า", "ำ", "ิ", "ี", "ึ", "ื", "ุ", "ู", "เ", "แ", "โ", "ใ", "ไ", "ๅ", "็", "่", "้", "๊", "๋", "์", 

In [18]:
texts[texts["sentence"] == ""]

,path,sentence


In [19]:
texts

,path,sentence
0,common_voice_th_25695281.mp3,ใครเป็นผู้รับ
1,common_voice_th_25695038.mp3,ผู้ชายคือช้างเท้าหน้า แต่ผู้หญิงคือควาญช้าง
2,common_voice_th_25701200.mp3,ไม่ต้องสงสัยเลย เธอเป็นคนเขียนจดหมายนี้
3,common_voice_th_25768746.mp3,คุณคือคนที่ดีที่สุด คุณผู้หญิง เขาพูดด้วยท่าที...
4,common_voice_th_25663832.mp3,ลองชิมสลัดนี้ดูสิ
...,...,...
107693,common_voice_th_27407562.mp3,คลองดำเนินสะดวก ในรายการบอกว่ามีสองประโยชน์คือ...
107694,common_voice_th_27407567.mp3,ผู้ที่เคยมีประวัติการแท้ง
107695,common_voice_th_27407592.mp3,ต่อมาที่แห่งนี้จึงได้ชื่อว่ากุดนางใย ใยหมายถึง...
107696,common_voice_th_27407619.mp3,ไม่ต้องเข้าโอเอสหลัก


In [20]:
#count duplicated sentences
texts.sentence.value_counts()

รู้สึกอย่างไร ฉันถาม                              15
ตอนนี้คุณรู้สึกอย่างไร                            14
ไม่มีอะไรให้ดู                                    14
ฉันจะรู้ได้อย่างไร                                13
เกิดอะไรขึ้นกับคุณ                                13
                                                  ..
บางกอก โพสต์                                       1
บัญชีเงินฝาก                                       1
มักมีเหตุขัดข้อง                                   1
เครือข่ายประสาทเทียมสามารถทำสิ่งที่คล้ายกันมาก     1
เอสไอเอสบี                                         1
Name: sentence, Length: 42014, dtype: int64

In [21]:
#count the dupliation
texts.sentence.value_counts().value_counts()

2     15067
1      9450
3      7273
4      6688
5      2633
6       449
7       163
8       159
9        86
10       27
11       10
12        4
14        2
13        2
15        1
Name: sentence, dtype: int64

In [23]:
#42k unique sentences out of 108k utterances
texts.shape, texts.sentence.nunique()

((107698, 2), 42014)

In [24]:
texts.to_csv('../data/validated_cleaned.tsv', index=False, sep='\t')